In [6]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, LSTM, MaxPooling2D,AveragePooling2D,GlobalMaxPooling2D, GlobalAveragePooling2D, Flatten, Dropout, Reshape, BatchNormalization, ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorboard
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler

from functools import partial
from matplotlib import rc, style
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
from scipy import stats

import sys, os, math, time, datetime


# reduce number of threads
# os.environ['TF_NUM_INTEROP_THREADS'] = '1' 
# os.environ['TF_NUM_INTRAOP_THREADS'] = '1' 

# disable GPU and anable MKL OneDNN
tf.config.set_visible_devices([], 'GPU')
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
os.environ['DNNL_VERBOSE'] = '0'

style.use("seaborn")
pd.plotting.register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale = 1)

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
# tf.compat.v1.disable_eager_execution()

LOOP_NUMBER=3

def shifting(bitlist):
    out = 0
    for bit in bitlist:
        out = (out << 1) | bit
    return out

def fromBit( b ) :
    if b == 0.0 :
        return -0.9
    return 0.9

def isCorrect( target, actual ) :
    if target < 0.0 :
        y1 = False
    else :
        y1 = True
    if actual < 0.0 :
        y2 = False
    else :
        y2 = True
    return y1 == y2

print(tf.__version__)

2.6.0


In [7]:
results_dir = "./results/mse_07102021_ES/"
results_list = os.listdir(results_dir)

for results_item in results_list:
    resultpath = results_dir + results_item
    display(f'Starting with {resultpath}')
    result = pd.read_csv(resultpath, index_col=0)
    dataset_no = result.loc['dataset_no'].values[0].split('_')
    dataset_name = f'seqnetdata.ni={dataset_no[1]}.no={dataset_no[2]}.mc={dataset_no[3]}.numTimeSteps{dataset_no[4]}.version{dataset_no[5]}.{dataset_no[6]}.csv'
    af_LSTM = result.loc['af_LSTM'].values[0]
    raf_LSTM = result.loc['raf_LSTM'].values[0]
    af_dense = result.loc['af_dense'].values[0]
    optimizer_dense = result.loc['optimizer'].values[0]
    learningrate = float(result.loc['learning_rate'].values[0])
    epoch_no = int(result.loc['tuner/epochs'].values[0])

    datapath = f'./Version9.128timesteps/{dataset_name}'
    display(datapath)
    with open(datapath, "r") as fp:
        [noInput, noOutput] = [int(x) for x in fp.readline().split(',')]
    rdf = np.array(pd.read_csv(datapath, skiprows=1))

    print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')
    df_val, df_train = train_test_split(rdf,test_size=0.5)
    print(df_train.shape, df_val.shape)

    print('Step 2: Separating values and labels.')
    # Training set
    x_train, y_train, x_val, y_val = None, None, None, None
    for i in range(df_train.shape[0]):
        df_train_i = df_train[i].reshape(-1, noInput+noOutput).astype('float32')
        x_train_i, y_train_i = df_train_i[:, 0:noInput], df_train_i[-1, noInput:]
        x_train = x_train_i[np.newaxis,:,:] if x_train is None else np.append(x_train, x_train_i[np.newaxis,:,:], axis=0)
        y_train = y_train_i.reshape(1, -1) if y_train is None else np.append(y_train, y_train_i.reshape(1, -1), axis=0)
    print("+ Training set:   ", x_train.shape, y_train.shape, x_train.dtype)

    # Validation set
    x_test, y_test = None, None
    for i in range(df_val.shape[0]):
        df_val_i = df_val[i].reshape(-1, noInput+noOutput).astype('float32')
        x_val_i, y_val_i = df_val_i[:, 0:noInput], df_val_i[-1, noInput:]
        x_val = x_val_i[np.newaxis,:,:] if x_val is None else np.append(x_val, x_val_i[np.newaxis,:,:], axis=0)
        y_val = y_val_i.reshape(1,-1) if y_val is None else np.append(y_val, y_val_i.reshape(1,-1), axis=0)
    print("+ Validation set: ", x_val.shape, y_val.shape, x_val.dtype)

    for i in range( x_train.shape[ 0 ] ) :
        for j in range( x_train.shape[ 1 ] ) :
            for k in range( x_train.shape[ 2 ] ) :
                x_train[ i, j, k ] = fromBit( x_train[ i, j, k ] )

    for i in range( y_train.shape[ 0 ] ) :
        for j in range( y_train.shape[ 1 ] ) :
            y_train[ i, j ] = fromBit( y_train[ i, j ] )

    for i in range( x_val.shape[ 0 ] ) :
        for j in range( x_val.shape[ 1 ] ) :
            for k in range( x_val.shape[ 2 ] ) :
                x_val[ i, j, k ] = fromBit( x_val[ i, j, k ] )

    for i in range( y_val.shape[ 0 ] ) :
        for j in range( y_val.shape[ 1 ] ) :
            y_val[ i, j ] = fromBit( y_val[ i, j ] )

    lstm_model = tf.keras.Sequential()
    lstm_model.add(
      tf.keras.layers.LSTM(
          units=8, 
          input_shape=[x_train.shape[1], x_train.shape[2]],
          activation=af_LSTM, recurrent_activation=raf_LSTM,
          unroll =False,
          use_bias=True,
          recurrent_dropout=0,
          return_sequences=False
      )
    )

    lstm_model.add(tf.keras.layers.Dense(y_train.shape[1], activation=af_dense))
    if(optimizer_dense == 'adam'):
        lstm_model.compile(loss='mse', optimizer=Adam(learning_rate = learningrate, decay=0), metrics=['mse'])
    else:
        lstm_model.compile(loss='mse', optimizer=SGD(learning_rate = learningrate), metrics=['mse'])
    
    for loop in range(LOOP_NUMBER):
        strt_time = datetime.datetime.now() 
        
        lstm_model.fit(
            x_train, y_train, 
            batch_size=1,
            verbose=1, # Suppress chatty output; use Tensorboard instead
            epochs=epoch_no,
            validation_data=(x_val, y_val),
            shuffle=True,
            callbacks=[tf.keras.callbacks.TerminateOnNaN()],
        )
        
        curr_time = datetime.datetime.now()
        timedelta = curr_time - strt_time
        dnn_train_time = timedelta.total_seconds()
        
        val_performance = lstm_model.evaluate(x_val, y_val)

        display(val_performance)
        y_pred = lstm_model.predict(x_val, verbose=1)

        count = 0
        numCorrect = 0
        for i in range( y_pred.shape[ 0 ] ) :
             for j in range( y_pred.shape[ 1 ] ) :
                count += 1
                if isCorrect( y_val[ i, j ], y_pred[ i, j ] ) :
                    numCorrect += 1

        print( f'Prediction_Accuracy_{loop} = {round(numCorrect / count,5)}' )
        result.loc[f'val_acc_{loop}'] = round(val_performance[1], 5)
        result.loc[f'pred_acc_{loop}'] = round(numCorrect / count, 5)
#     result.to_csv(resultpath, index=True)
    sys.exit()

'Starting with ./results/mse_07102021_ES/mse_3_5_30_128_9_2.csv'

'./Version9.128timesteps/seqnetdata.ni=3.no=5.mc=30.numTimeSteps128.version9.2.csv'

Step 1: Dividing the training and testing set with ratio 1:1 (50%).
(5000, 1024) (4999, 1024)
Step 2: Separating values and labels.
+ Training set:    (5000, 128, 3) (5000, 5) float32
+ Validation set:  (4999, 128, 3) (4999, 5) float32
Train on 5000 samples, validate on 4999 samples
Epoch 1/3
4997/5000 [============================>.] - ETA: 0s - loss: 0.8133 - mse: 0.8133

/home/ifeai/anaconda3/envs/py385_gpu/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


5000/5000 [==============================] - 79s 16ms/sample - loss: 0.8133 - mse: 0.8133 - val_loss: 0.8122 - val_mse: 0.8122
Epoch 2/3
5000/5000 [==============================] - 78s 16ms/sample - loss: 0.8108 - mse: 0.8108 - val_loss: 0.8111 - val_mse: 0.8111
Epoch 3/3
5000/5000 [==============================] - 78s 16ms/sample - loss: 0.8105 - mse: 0.8105 - val_loss: 0.8113 - val_mse: 0.8113


...

Prediction_Accuracy_1 = 0.49578
Train on 5000 samples, validate on 4999 samples
Epoch 1/3
5000/5000 [==============================] - 78s 16ms/sample - loss: 0.8097 - mse: 0.8097 - val_loss: 0.8113 - val_mse: 0.8113
Epoch 2/3
5000/5000 [==============================] - 78s 16ms/sample - loss: 0.8095 - mse: 0.8095 - val_loss: 0.8115 - val_mse: 0.8115
Epoch 3/3
5000/5000 [==============================] - 78s 16ms/sample - loss: 0.8093 - mse: 0.8093 - val_loss: 0.8116 - val_mse: 0.8116


[0.811591522637356, 0.8115912]

Prediction_Accuracy_2 = 0.50366


SystemExit: 

/home/ifeai/anaconda3/envs/py385_gpu/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
val_performance = lstm_model.evaluate(x_val, y_val)
display(val_performance)
y_pred = lstm_model.predict(x_val, verbose=1)

[0.811591522637356, 0.8115912]

In [18]:
mse = tf.keras.metrics.MeanSquaredError()
display(mse(y_val, y_pred))
display(y_val)
display(y_pred)

<tf.Tensor 'Identity_1:0' shape=() dtype=float32>

array([[ 0.9,  0.9,  0.9, -0.9, -0.9],
       [ 0.9, -0.9, -0.9,  0.9,  0.9],
       [ 0.9,  0.9, -0.9,  0.9, -0.9],
       ...,
       [-0.9, -0.9,  0.9,  0.9,  0.9],
       [ 0.9, -0.9,  0.9, -0.9, -0.9],
       [-0.9,  0.9, -0.9,  0.9, -0.9]], dtype=float32)

array([[ 0.03719394,  0.00917129,  0.00237269, -0.05457478, -0.06050435],
       [-0.00512276, -0.03423172,  0.03487341,  0.03129037, -0.05336945],
       [ 0.01684816, -0.05089641,  0.00243355,  0.03791634, -0.0042509 ],
       ...,
       [-0.03223786, -0.01613243, -0.01621141, -0.03601751,  0.00840194],
       [-0.01406464, -0.01610354, -0.00356473, -0.0168545 ,  0.01225847],
       [ 0.00308946, -0.01075396,  0.01961995,  0.00310873, -0.02409352]],
      dtype=float32)

In [15]:
tf.math.squared_difference(y_val, y_pred)

<tf.Tensor 'SquaredDifference:0' shape=(4999, 5) dtype=float32>